# zarr-ification of CESM-LE for AWS

In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import shutil

import yaml

import xarray as xr

import intake
from distributed.utils import format_bytes
from operator import mul
from functools import reduce

## Spinup cluster

In [2]:
from ncar_jobqueue import NCARCluster
cluster = NCARCluster(memory="70GB")
cluster

In [3]:
print(cluster.job_script())

#!/bin/bash

#!/usr/bin/env bash
#SBATCH -J dask-worker
#SBATCH -A NIOW0001
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=66G
#SBATCH -t 06:00:00
#SBATCH -C skylake
#SBATCH -C casper
JOB_ID=${SLURM_JOB_ID%;*}



/glade/work/abanihi/softwares/miniconda3/envs/analysis/bin/python -m distributed.cli.dask_worker tcp://10.12.203.5:39647 --nthreads 1 --memory-limit 70.00GB --name dask-worker--${JOB_ID}-- --death-timeout 60 --interface ib0



In [4]:
cluster.adapt(minimum=0, maximum=30, wait_count=60)

In [5]:
from dask.distributed import Client
client = Client(cluster) # Connect this local process to remote workers
client

Client Scheduler: tcp://10.12.203.5:39647 Dashboard: https://jupyterhub.ucar.edu/ch/user/abanihi/proxy/8787/status,Cluster Workers: 30 Cores: 30 Memory: 2.10 TB


## Connect to intake-esm database

In [6]:
col = intake.open_esm_metadatastore(collection_name='CESM1-LE')
col.df.head()

,resource,resource_type,direct_access,experiment,case,component,stream,variable,date_range,member_id,file_fullpath,file_basename,file_dirname,ctrl_branch_year,year_offset,sequence_order,has_ocean_bgc,grid
0,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,18000101-18991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.180001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
1,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,08000101-08991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.080001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
2,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,04020101-04991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.040201...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
3,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,10000101-10991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.100001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN
4,CTRL:GLADE:posix:/glade/collections/cdg/data/c...,posix,True,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,T200,11000101-11991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,b.e11.B1850C5CN.f09_g16.005.cam.h1.T200.110001...,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0,0,True,NaN


## Helper functions

In [7]:
def collection_summary(col):
    df = col.df
    print("There are:\n")
    print(f"\t1) {df.experiment.nunique()} experiments: \n\n\t\t{df.experiment.unique()}\n")
    print(f"\t2) {df.case.nunique()} cases: \n\n\t\t{df.case.unique()[:5]} etc...\n")
    print(f"\t3) {df.component.nunique()} components: \n\n\t\t{df.component.unique()}\n")
    print(f"\t4) {df.stream.nunique()} streams: \n\n\t\t{df.stream.unique()}\n")
    print(f"\t5) {df.variable.nunique()} variables: \n\n\t\t{df.variable.unique()[:10]} etc...\n")
    

In [8]:
def print_query_info(query_results):
    variables = sorted(query_results.variable.unique().tolist())
    files = sorted(query_results.file_fullpath.unique().tolist())
    date_ranges = sorted(query_results.date_range.unique())
    cases = sorted(query_results.case.unique())
    members = sorted(query_results.member_id.unique())
    print(f"This query: {query} returned:\n\n- {len(variables)} unique variable(s)\n\n")
    print(f"- Number of files: {len(files)}\n\n")
    print(f"- {len(members)} member(s): {members}\n\n")
    #print(f"Date ranges: {date_ranges}\n\n")
    print(f"- {len(cases)} case(s)\n\n")
    

In [9]:
# file namer
dirout = '/glade/scratch/abanihi/calcs'
def zarr_file(exp, cmp, stream, var, write=False):
    path = f'{dirout}/CESM-LE-{exp}-{cmp}-{stream}-{var}.zarr'    
    if write and os.path.exists(path):
        shutil.rmtree(path)
    print(path)
    return path     

In [10]:
def read_data(cat, xr_open):
    dd = cat.to_xarray(**xr_open)
    _, ds = dd.popitem()
    return ds.chunk({'member_id': 4})

def print_ds_info(ds, var):
    dt = ds[var].dtype
    itemsize = dt.itemsize
    chunk_size = ds[var].data.chunksize
    size = format_bytes(ds.nbytes)
    _bytes = reduce(mul, chunk_size) * itemsize
    chunk_size_bytes = format_bytes(_bytes)
    
    print(f"Variable name: {var}")
    print(f"Chunk shape: {chunk_size}")
    print(f"Dataset shape: {ds[var].shape}")
    print(f"Chunk size: {chunk_size_bytes}")
    print(f"Dataset size: {size}")

In [11]:
collection_summary(col)

There are:

	1) 6 experiments: 

		['CTRL' 'CTRL-AMIP' 'CTRL-SLAB-OCN' '20C' '20C-OIC' 'RCP85']

	2) 93 cases: 

		['b.e11.B1850C5CN.f09_g16.005' 'f.e11.F1850C5CN.f09_f09.001'
 'e.e11.E1850C5CN.f09_g16.001' 'b.e11.B20TRC5CNBDRD.f09_g16.105'
 'b.e11.B20TRC5CNBDRD.f09_g16.104'] etc...

	3) 5 components: 

		['atm' 'ocn' 'lnd' 'rof' 'ice']

	4) 10 streams: 

		['cam.h1' 'cam.h0' 'pop.h.nday1' 'pop.h' 'clm2.h1' 'clm2.h0' 'rtm.h1'
 'rtm.h0' 'cice.h1' 'cice.h']

	5) 985 variables: 

		['T200' 'Q200' 'FLNSC' 'Q500' 'PRECL' 'QBOT' 'TREFHTMN' 'FSNTOA' 'TAUX'
 'TAUY'] etc...



## Load variable/chunking specs

In [12]:
with open("../variables_chunking_spec.yaml") as f:
    specs = yaml.safe_load(f)
specs

{'ocn': {'pop.h': {'variable_category': {'2D': {'variables': ['SST', 'SSH'],
     'chunks': {'time': 120, 'nlat': 192, 'nlon': 160}},
    '3D': {'variables': ['SALT'],
     'chunks': {'time': 120, 'z_t': 6, 'nlat': 192, 'nlon': 160}}},
   'frequency': 'monthly'}},
 'ice': {'cice.h': {'variable_category': {'2D': {'variables': ['aice_nh',
      'aice_sh',
      'hi_nh',
      'hi_sh'],
     'chunks': {'time': 240}}},
   'frequency': 'monthly'}},
 'atm': {'cam.h0': {'variable_category': {'2D': {'variables': ['T',
      'U',
      'v',
      'Q',
      'Z3'],
     'chunks': {'time': 120, 'lat': 96, 'lon': 72}},
    '3D': {'variables': ['FLNS',
      'FLNSC',
      'FLUT',
      'FSNS',
      'FSNSC',
      'FSNTOA',
      'ICEFRAC',
      'LHFLX',
      'PRECC',
      'PRECL',
      'PRECSC',
      'PRECSL',
      'PSL',
      'SHFLX',
      'TMQ',
      'TREFHT',
      'TREFHTMN',
      'TREFHTMX',
      'TS'],
     'chunks': {'time': 120}}},
   'frequency': 'monthly'}},
 'lnd': {'clm2.h0

## Process `land` component (realm)

In [13]:
component_list = ['lnd']

experiment_list = ['20C', 'RCP85']

In [14]:
for exp in experiment_list:
    for cmp in component_list:
        for stm, stm_specs in specs[cmp].items():
            frequency = stm_specs['frequency']
            var_cats = stm_specs['variable_category']
            for var_cat, var_cat_values in var_cats.items():
                variables = var_cat_values['variables']
                chunks = var_cat_values['chunks']
                xr_open = dict(chunks=chunks, decode_times=True, decode_coords=False)
                for variable in variables:
                    query = dict(experiment=exp, component=cmp, stream=stm, variable=variable)
                    cat = col.search(**query)
                    results = cat.query_results
                    if not results.empty:
                        print("*"*100)
                        print_query_info(results)
                        try:
                            ds = read_data(cat, xr_open)
                            print("+"*35)
                            if variable in {'aice_nh', 'aice_sh'}:
                                variable = 'aice'
                            elif variable in {'hi_nh', 'hi_sh'}:
                                variable = 'hi'
                            print_ds_info(ds, variable)
                            print("+"*35)
                            print(ds)
                            print("*"*100)
                        except Exception as e:
                            raise(e)
                    else:
                        print(f"No results found for query={query}")
                        continue

****************************************************************************************************
This query: {'experiment': '20C', 'component': 'lnd', 'stream': 'clm2.h0', 'variable': 'FSNO'} returned:

- 1 unique variable(s)


- Number of files: 40


- 40 member(s): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 101, 102, 103, 104, 105]


- 40 case(s)





+++++++++++++++++++++++++++++++++++
Variable name: FSNO
Chunk shape: (4, 480, 96, 144)
Dataset shape: (40, 1872, 192, 288)
Chunk size: 106.17 MB
Dataset size: 17.43 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1872)
Coordinates:
  * time          (time) object 1850-02-01 00:00:00 ... 2006-01-01 00:00:00
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3, 96, 144)>



+++++++++++++++++++++++++++++++++++
Variable name: H2OSNO
Chunk shape: (4, 480, 96, 144)
Dataset shape: (40, 1872, 192, 288)
Chunk size: 106.17 MB
Dataset size: 17.43 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1872)
Coordinates:
  * time          (time) object 1850-02-01 00:00:00 ... 2006-01-01 00:00:00
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3, 96, 144)


+++++++++++++++++++++++++++++++++++
Variable name: QRUNOFF
Chunk shape: (4, 480, 96, 144)
Dataset shape: (40, 1872, 192, 288)
Chunk size: 106.17 MB
Dataset size: 17.43 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1872)
Coordinates:
  * time          (time) object 1850-02-01 00:00:00 ... 2006-01-01 00:00:00
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3, 96, 144


+++++++++++++++++++++++++++++++++++
Variable name: RAIN
Chunk shape: (4, 480, 96, 144)
Dataset shape: (40, 1872, 192, 288)
Chunk size: 106.17 MB
Dataset size: 17.43 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1872)
Coordinates:
  * time          (time) object 1850-02-01 00:00:00 ... 2006-01-01 00:00:00
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3, 96, 144)>


...


+++++++++++++++++++++++++++++++++++
Variable name: QRUNOFF
Chunk shape: (4, 480, 96, 144)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.95 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3, 96, 144


+++++++++++++++++++++++++++++++++++
Variable name: RAIN
Chunk shape: (4, 480, 96, 144)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.95 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3, 96, 144)>



+++++++++++++++++++++++++++++++++++
Variable name: SNOW
Chunk shape: (4, 480, 96, 144)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.95 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3, 96, 144)>



+++++++++++++++++++++++++++++++++++
Variable name: SOILWATER_10CM
Chunk shape: (4, 480, 96, 144)
Dataset shape: (40, 1140, 192, 288)
Chunk size: 106.17 MB
Dataset size: 10.95 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:         (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat             (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd         (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak          (levlak) float32 0.05 0.6 2.1 4.6 ... 25.6 34.325 44.775
  * lon             (lon) float32 0.0 1.25 2.5 3.75 ... 356.25 357.5 358.75
  * time            (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id       (member_id) int64 1 2 3 4 5 6 7 ... 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW             (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3


+++++++++++++++++++++++++++++++++++
Variable name: SOILLIQ
Chunk shape: (4, 480, 3, 96, 144)
Dataset shape: (40, 1140, 15, 192, 288)
Chunk size: 318.50 MB
Dataset size: 152.15 GB
+++++++++++++++++++++++++++++++++++
<xarray.Dataset>
Dimensions:       (hist_interval: 2, lat: 192, levgrnd: 15, levlak: 10, lon: 288, member_id: 40, time: 1140)
Coordinates:
  * lat           (lat) float32 -90.0 -89.057594 -88.11518 ... 89.057594 90.0
  * levgrnd       (levgrnd) float32 0.007100635 0.027925 ... 21.32647 35.17762
  * levlak        (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.325 44.775
  * lon           (lon) float32 0.0 1.25 2.5 3.75 ... 355.0 356.25 357.5 358.75
  * time          (time) object 2006-02-01 00:00:00 ... 2101-01-01 00:00:00
  * member_id     (member_id) int64 1 2 3 4 5 6 7 ... 34 35 101 102 103 104 105
Dimensions without coordinates: hist_interval
Data variables:
    BSW           (member_id, levgrnd, lat, lon) float32 dask.array<shape=(40, 15, 192, 288), chunksize=(4, 3,